## SoS Workflow:

This is the options and the SoS code to run the LDSC pipeline using your own data. 

## Command Interface:

In [1]:
!sos run LDSC_DeepSea_Code.ipynb -h

usage: sos run LDSC_DeepSea_Code.ipynb
               [workflow_name | -t targets] [options] [workflow_options]
  workflow_name:        Single or combined workflows defined in this script
  targets:              One or more targets to generate
  options:              Single-hyphen sos parameters (see "sos run -h" for details)
  workflow_options:     Double-hyphen workflow-specific parameters

Workflows:
  train_model
  make_annot
  format_annot
  munge_sumstats_no_sign
  munge_sumstats_sign
  calc_ld_score
  calc_enrichment

Sections
  train_model:
  make_annot:
  format_annot:
    Workflow Options:
      --full-annot VAL (as str, required)
                        path to full annotation file
      --output VAL (as str, required)
                        path to output file directory
  munge_sumstats_no_sign: This option is for when the summary statistic file
                        does not contain a signed summary statistic (Z or Beta).
                        In this case,the program

## Train Model:

In [93]:

[train_model]

bash: container='/mnt/mfs/statgen/Anmol/deepsea_latest.sif'

    python3.7 /mnt/mfs/statgen/Anmol/training_files/run_neuron_full_pipeline.py 

## Make Full Annotation File Based on Trained Model

In [ ]:

[make_annot]

bash: container='/mnt/mfs/statgen/Anmol/deepsea_latest.sif'

    python3.7 /mnt/mfs/statgen/Anmol/training_files/variant_pred_pipeline.py

## Format Annotation File

In [ ]:

[format_annot]

#path to full annotation file
parameter: full_annot = str
#path to output file directory
parameter: output = str

R: container="/mnt/mfs/statgen/Anmol/r-packages.sif"
    library(data.table)
    library(tidyverse)
    data = fread({full_annot})
    data$V1 = gsub("chr","",data$V1)
    data$V1 = as.numeric(data$V1)
    features = colnames(data)[9:ncol(data)]
    features = data.frame(features)
    features$encoding = paste0("feat_",seq(1,nrow(features)))
    fwrite(features,paste0({output},"feature_encoding.txt"),quote=F,sep="\t",row.names=F,col.names=T)
    for (i in seq(1,22)){
    data_2 = filter(data,V1==i)
    data_2 = select(data_2,-c(seq(4,8)))
    for (j in seq(4,ncol(data_2))){
    data_3 = select(data_2,c("V1","V2","V3",j))
    data_3$CM = 0
    colnames(data_3) = c("CHR","BP","SNP",paste0("feat_",j),"CM")
    fwrite(data_3,paste0({output},"chr_",j,".annot.gz"),quote=F,sep="\t",row.names=F,col.names=T)
    }
    }

## Munge Summary Statistics (Option 1: No Signed Summary Statistic):

In [ ]:
#This option is for when the summary statistic file does not contain a signed summary statistic (Z or Beta). 
#In this case,the program will calculate Z for you based on A1 being the risk allele
[munge_sumstats_no_sign]



#path to summary statistic file
parameter: sumst = str
#path to Hapmap3 SNPs file, keep all columns (SNP, A1, and A2) for the munge_sumstats program
parameter: alleles = "w_hm3.snplist"
#path to output file
parameter: output = str

bash: 
    python2 munge_sumstats.py --sumstats {sumst} --merge-alleles {alleles} --out {output} --a1-inc

## Munge Summary Statistics (Option 2: No Signed Summary Statistic):

In [ ]:
# This option is for when the summary statistic file does contain a signed summary statistic (Z or Beta)
[munge_sumstats_sign]



#path to summary statistic file
parameter: sumst = str
#path to Hapmap3 SNPs file, keep all columns (SNP, A1, and A2) for the munge_sumstats program
parameter: alleles = "w_hm3.snplist"
#path to output file
parameter: output = str

bash: 
    python2 munge_sumstats.py --sumstats {sumst} --merge-alleles {alleles} --out {output}

## Calculate LD Scores:

**Make sure to delete SNP,CHR, and BP columns from annotation files if they are present otherwise this code will not work. Before deleting, if these columns are present, make sure that the annotation file is sorted.**

In [ ]:
#Calculate LD Scores
#**Make sure to delete SNP,CHR, and BP columns from annotation files if they are present otherwise this code will not work. Before deleting, if these columns are present, make sure that the annotation file is sorted.**
[calc_ld_score]

#Path to bim file
parameter: bim = str
#Path to annotation File. Make sure to remove the SNP, CHR, and BP columns from the annotation file if present before running.
parameter: annot_file = str
#name of output file
parameter: output = str
#path to Hapmap3 SNPs file, remove the A1 and A2 columns for the Calculate LD Scores program 
parameter: snplist = "w_hm3.snplist"

bash: 
    python2 ldsc.py --bfile {bim} --l2 --ld-wind-cm 1 --annot {annot_file} --thin-annot --out {output} --print-snps {snplist}

## Calculate Functional Enrichment using Annotations:

In [ ]:
#Calculate Enrichment Scores for Functional Annotations

[calc_enrichment]

#Path to Summary statistics File
parameter: sumstats = str
#Path to Reference LD Scores Files (Base Annotation + Annotation you want to analyze, format like minimal working example)
parameter: ref_ld = str
#Path to LD Weight Files (Format like minimal working example)
parameter: w_ld = str
#path to frequency files (Format like minimal working example)
parameter: frq_file = str
#Output name
parameter: output = str

bash:
    python2 ldsc.py --h2 {sumstats} --ref-ld-chr {ref_ld} --w-ld-chr {w_ld} --overlap-annot --frqfile-chr {frq_file} --out {output}